In [7]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

Let's adjust the **dilation factor**, **kernel sizes**, and **stride** to bring the receptive field down closer to your target range.

### Adjustments:

* **Dilation**: We can reduce the dilation factor to **2** to shrink the receptive field while still using dilation to maintain the global context.
* **Kernel Size**: We’ll use smaller kernel sizes than (7 \times 7) in some layers to reduce the receptive field growth rate.
* **Stride**: We’ll make sure that the strides don’t downsample the feature maps too aggressively, keeping more spatial information at later stages.

Let’s aim to use a **dilation of 2** (instead of 3) and use **(5 \times 5) kernels** for some layers, instead of the larger (7 \times 7). We will also adjust strides carefully.

### Updated Network Design (Receptive Field: ~44–50)

```python

```

### Key Changes:

1. **Dilation = 2**: I reduced the dilation to 2 to control the growth of the receptive field. Dilation of 2 still provides a nice expansion without overshooting the target receptive field.
2. **Smaller Kernels ((5 \times 5))**: The kernel sizes are reduced to (5 \times 5) to control how much the receptive field expands at each layer. The larger the kernel, the larger the receptive field, so we keep it smaller to prevent overshooting.
3. **Stride = 2**: I kept stride as 2 for downsampling after the first couple of convolution blocks. This keeps the feature map shrinking at a moderate rate.
4. **Padding**: Adjusted padding to ensure that the convolution doesn't shrink the feature map dimensions too quickly. Since we are using dilation, we need to match the padding accordingly.

---

### Receptive Field Calculation:

Let’s now calculate the receptive field layer-by-layer for this architecture:

#### **Conv Block 1** (with (5 X 5) kernels, dilation=2):

* **First layer**: (5 s 5), dilation=2 → RF = (5 + (5 - 1) X 2 = 13)
* **Second layer**: (5 x 5), dilation=2, stride=2 → RF = (13 + (5 - 1) X 2 = 21)

#### **Conv Block 2** (with (5 X 5) kernels, dilation=2):

* **First layer**: (5 X 5), dilation=2 → RF = (21 + (5 - 1) X 2 = 29)
* **Second layer**: (5 X 5), dilation=2, stride=2 → RF = (29 + (5 - 1) X 2 = 37)

#### **Conv Block 3** (with (5 X 5) kernels, dilation=2):

* **First layer**: (5 X 5), dilation=2 → RF = (37 + (5 - 1) \times 2 = 45)

#### **Conv Block 4** (Depthwise convolution with dilation=2):

* **Depthwise convolution layer**: (3 X 3), dilation=2 → RF = (45 + (3 - 1) \times 2 = 49)
* **Pointwise convolution**: (1 X 1) kernel → RF = **49** (no further change).

---

### Final Receptive Field:

The receptive field is **49x49**, which falls within your desired range of **44 to 50**.

### Summary of Adjustments:

* **Receptive Field**: Now close to **49x49**, which is just under the 50 mark.
* **Parameters**: The number of parameters has remained relatively constant because we used (5 \times 5) kernels and dilation rather than larger kernels or more complex operations.
* **Stride**: The stride was kept at 2 in some layers to avoid excessive downsampling, maintaining feature map dimensions.




In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
dropout_value = 0.0001

class ciferNet(nn.Module):
    def __init__(self, dilation=2):
        super(ciferNet, self).__init__()

        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5, 5), padding=2, dilation=dilation, bias=False),  # Kernel size: 5x5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding=2, dilation=dilation, bias=False),  # Kernel size: 5x5, Changed stride to 1
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # nn.Dropout(dropout_value),
        )

        # Conv Block 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=2, dilation=dilation, bias=False),  # Kernel size: 5x5
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=1, padding=2, dilation=dilation, bias=False),  # Kernel size: 5x5, Changed stride to 1
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # nn.Dropout(dropout_value),
        )

        # Conv Block 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=4, dilation=dilation, bias=False),  # Kernel size: 5x5, Adjusted padding
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # nn.Dropout(dropout_value),
        )

        # Conv Block 4 → Depthwise separable convolution with dilation
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=2, dilation=dilation, groups=128, bias=False),  # Depthwise conv with dilation
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False),  # Pointwise conv
            nn.ReLU(),
            nn.BatchNorm2d(256),
            # nn.Dropout(dropout_value),
        )

        # GAP and classifier
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)

In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]           1,200
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,608
              ReLU-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
            Conv2d-7           [-1, 64, 28, 28]          18,432
              ReLU-8           [-1, 64, 28, 28]               0
       BatchNorm2d-9           [-1, 64, 28, 28]             128
           Conv2d-10           [-1, 64, 28, 28]          36,864
             ReLU-11           [-1, 64, 28, 28]               0
      BatchNorm2d-12           [-1, 64, 28, 28]             128
           Conv2d-13          [-1, 128, 32, 32]          73,728
             ReLU-14          [-1,

In [14]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Train Phase
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.0625,
        scale_limit=0.1,
        rotate_limit=45,
        p=0.5
    ),
    A.CoarseDropout(
        max_holes=1,
        max_height=16,
        max_width=16,
        min_holes=1,
        min_height=16,
        min_width=16,
        fill_value=(123.68, 116.78, 103.94),
        mask_fill_value=None,
        p=0.5
    ),
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])

# Test Phase
test_transforms = A.Compose([
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-306781532.py:13: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [15]:
from torch.utils.data import Dataset

class CIFAR10Albumentations(Dataset):
    def __init__(self, root, train=True, download=False, transform=None):
        self.cifar10 = datasets.CIFAR10(root=root, train=train, download=download)
        self.transform = transform

    def __getitem__(self, index):
        image, label = self.cifar10[index]
        if self.transform:
            # Albumentations requires the image to be passed as a keyword argument 'image'
            image = self.transform(image=np.array(image))['image'] # Convert PIL to numpy array before applying transform
        return image, label

    def __len__(self):
        return len(self.cifar10)

In [16]:
import numpy as np

train = CIFAR10Albumentations('./data', train=True, download=True, transform=train_transforms)
test = CIFAR10Albumentations('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 170M/170M [00:03<00:00, 48.1MB/s]


In [17]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? True


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [18]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [19]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 35
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.9639126062393188 Batch_id=390 Accuracy=35.85: 100%|██████████| 391/391 [00:27<00:00, 14.47it/s]



Test set: Average loss: 1.9218, Accuracy: 4575/10000 (45.75%)

EPOCH: 1


Loss=1.5883114337921143 Batch_id=390 Accuracy=46.60: 100%|██████████| 391/391 [00:26<00:00, 14.68it/s]



Test set: Average loss: 1.4829, Accuracy: 5295/10000 (52.95%)

EPOCH: 2


...


Test set: Average loss: 0.6369, Accuracy: 7866/10000 (78.66%)

EPOCH: 33


Loss=0.5861379504203796 Batch_id=390 Accuracy=77.10: 100%|██████████| 391/391 [00:26<00:00, 14.97it/s]



Test set: Average loss: 0.6231, Accuracy: 7919/10000 (79.19%)

EPOCH: 34


Loss=0.8220704793930054 Batch_id=390 Accuracy=77.06: 100%|██████████| 391/391 [00:26<00:00, 14.95it/s]



Test set: Average loss: 0.6185, Accuracy: 7912/10000 (79.12%)

